In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pickle
from scipy.stats import skew, kurtosis, pearsonr
from scipy.signal import butter, welch, filtfilt, resample
import math
import nolds
import time
from PreprocessFcns import *
%matplotlib inline

In [2]:
import sklearn

In [3]:
sklearn.__version__

'0.21.2'

In [4]:
time.sleep(3600*.5)

In [5]:
full_dict = {}
count = 0
subj_list = (['Subject1'])
for subj in subj_list:
    f = open(subj + '.pkl', 'rb') # open pickle file containing dictionary of raw data for this subject
    full_dict[subj] = (pickle.load(f))
    f.close()
    count = count + 1
    if count < 10:
        print(' ' + str(count) + ' of '+ str(len(subj_list)) +' subjects loaded')
    else:
        print(str(count) + ' of '+ str(len(subj_list)) +' subjects loaded')

 1 of 20 subjects loaded
 2 of 20 subjects loaded
 3 of 20 subjects loaded
 4 of 20 subjects loaded
 5 of 20 subjects loaded
 6 of 20 subjects loaded
 7 of 20 subjects loaded
 8 of 20 subjects loaded
 9 of 20 subjects loaded
10 of 20 subjects loaded
11 of 20 subjects loaded
12 of 20 subjects loaded
13 of 20 subjects loaded
14 of 20 subjects loaded
15 of 20 subjects loaded
16 of 20 subjects loaded
17 of 20 subjects loaded
18 of 20 subjects loaded
19 of 20 subjects loaded
20 of 20 subjects loaded


In [6]:
full_dict['Subject1'].keys()

dict_keys(['Motor #1: Standing', 'Motor #2: Walking', 'Motor #3: Walking while Counting', 'Motor #4: Finger to Nose', 'Motor #5: Alternating Hand Movements', 'Motor #6: Sit to Stand', 'Motor #7: Drawing on Paper', 'Motor #8: Typing on a Computer', 'Motor #9: Nuts and Bolts', 'Motor #10: Drinking Water', 'Motor #11: Organizing Folder', 'Motor #12: Folding Towels', 'Motor #13: Sitting', 'Motor #4: Finger to Nose - Right', 'Motor #4: Finger to Nose - Left', 'Motor #5: Alternating Hand Movements - Right', 'Motor #5: Alternating Hand Movements - Left'])

In [7]:
#save corrected version of motor scores.xls 
mot_scores = pd.read_excel('MotorTasks.xls') #File containing clinician scores for tasks

In [8]:
def DataAggregatorAccelGyro(full_dict,subj_dict,task_list,task_scores,session_list,freq):
    
    Data = pd.DataFrame()
    
    #dataframe to track lost/missing data
    numSamples = pd.DataFrame()
    
    #Calculate mean across all scores for model comparison
    count = 0
    bradyTotal = 0
    tremorTotal = 0
    
    s = 0
    NAccel = 0; NGyro = 0
    ts_accel = []; ts_gyro = []
    for subj in subj_dict.keys():
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_dict)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            side = subj_dict[subj]
                
            loc = 'dorsal_hand_' + side

#             if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
#                 task_score = ts + ' ' + side

            if side == 'left':
                otherside = 'right'
            else:
                otherside = 'left'

            notBrady = ['standing', 'sitting', 'sit to stand', 'alternating '+otherside, 'finger to nose '+otherside]

            if (task_score not in notBrady):
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                        task_score+ ' ' + 'tremor ' + side + ' upper limb',
                                        task_score+ ' ' + 'bradykinesia ' + side + ' upper limb']]
            else:
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                        task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score[task_score+ ' ' + 'bradykinesia ' + side + ' upper limb'] = np.nan
                
            subj_score = subj_score.rename(index=str,columns={subj_score.columns[3]:'Bradykinesia',subj_score.columns[2]:'Tremor'})
            subj_score.index = range(len(subj_score))

            n_visits = len(subj_score)
            n_recordings = len(act_dict[task].keys())
            N = n_visits

            if n_visits != n_recordings:
                #print('Recordings error')
                N = min([n_visits,n_recordings])

            for trial in range(N):
                
                sensor = 'accel'
                    
                if len(act_dict[task][trial][loc][sensor]) <= 6:
                    continue

                #ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                #act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                data = HPfilter(act_dict[task][trial][loc][sensor])

                clip_data = gen_clips_mc10(data,downsample=freq,basefreq=62.5)

                ts, N = feature_extraction(clip_data)
                ts_accel.append(np.vstack([np.array(t) for t in ts]))
                NAccel+=N
#                 print(ts_accel)
                
                featcols = []

                if 'features' in clip_data.keys():
                    A = clip_data['features']
                    for c in range(len(A.columns)):
                        featcols.append(A.columns[c] + ' acc')
                    
                sensor = 'gyro'
                    
                if len(act_dict[task][trial][loc][sensor]) <= 6:
                    continue
                    
                clip_data = gen_clips_mc10(act_dict[task][trial][loc][sensor],downsample=freq,basefreq=62.5)
                
                ts, N = feature_extraction(clip_data)
                ts_gyro.append(np.vstack([np.array(t) for t in ts]))
                NGyro+=N
#                 print(ts_gyro)
                
                if 'features' in clip_data.keys():
                    G = clip_data['features']
                    for c in range(len(G.columns)):
                        featcols.append(G.columns[c] + ' gyr')
                
                D = pd.concat([A,G],axis=1)
                D.columns = featcols
                D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                D['Tremor'] = subj_score['Tremor'][trial]
                D['Visit'] = session_list[trial]
                D['Task'] = task
                D['Subject'] = subj
                Data = pd.concat([Data,D])
                    
                #clip_lens = []    
                
                #for c in range(len(clip_data['data'])):
                #    clip_lens.append(len(clip_data['data'][c]))
                #N = pd.DataFrame(data=np.asarray(clip_lens),columns=['Samples'],dtype='int')
                #N['Subject'] = subj
                #N['Task'] = task
                #N['Visit'] = session_list[trial]
                #numSamples = pd.concat([numSamples,N])
                                                
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    
    #numSamples = numSamples[['Subject','Visit','Task','Samples']]
    
    #if freq==62.5:
    #    numSamples.to_csv('Z:CIS-PD Study\\Sensor Sample Lengths.csv')
    
    return Data, (ts_accel,NAccel), (ts_gyro,NGyro)

In [9]:
def DataAggregatorAccelOnly(full_dict,subj_dict,task_list,task_scores,session_list,freq):

    sensor = 'accel'
    
    Data = pd.DataFrame()
    
    #dataframe to track lost/missing data
    numSamples = pd.DataFrame()
    
    #Calculate mean across all scores for model comparison
    count = 0
    bradyTotal = 0
    tremorTotal = 0
    
    s = 0
    for subj in subj_dict.keys():
        
        act_dict = full_dict[subj]
        
        s += 1
        print('Subject %d (%d of %d)'%(subj,s,len(subj_dict)))
        
        for t in range(len(task_list)):
            
            task = task_list[t]
            task_score = task_scores[t]
            ts = task_score
            
            side = subj_dict[subj]
                
            loc = 'dorsal_hand_' + side

#             if (task_score.find('alternating') > -1) | (task_score.find('finger to nose') > -1):
#                 task_score = ts + ' ' + side

            if side == 'left':
                otherside = 'right'
            else:
                otherside = 'left'

            notBrady = ['standing', 'sitting', 'sit to stand', 'alternating '+otherside, 'finger to nose '+otherside]

            if (task_score not in notBrady):
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                        task_score+ ' ' + 'tremor ' + side + ' upper limb',
                                        task_score+ ' ' + 'bradykinesia ' + side + ' upper limb']]
            else:
                subj_score = mot_scores.loc[mot_scores['subject']==subj,['subject','visit',
                                        task_score+ ' ' + 'tremor ' + side + ' upper limb']]
                subj_score[task_score+ ' ' + 'bradykinesia ' + side + ' upper limb'] = np.nan
                
            subj_score = subj_score.rename(index=str,columns={subj_score.columns[3]:'Bradykinesia',subj_score.columns[2]:'Tremor'})
            subj_score.index = range(len(subj_score))

            n_visits = len(subj_score)
            n_recordings = len(act_dict[task].keys())
            N = n_visits

            if n_visits != n_recordings:
                #print('Recordings error')
                N = min([n_visits,n_recordings])

            for trial in range(N):
                    
                if len(act_dict[task][trial][loc][sensor]) <= 6:
                    continue

                ZInds = np.diff(act_dict[task][trial][loc][sensor].index)!=0
                act_dict[task][trial][loc][sensor] = act_dict[task][trial][loc][sensor][np.hstack((ZInds,np.array(True)))]

                data = HPfilter(act_dict[task][trial][loc][sensor])

                clip_data = gen_clips_mc10(data,downsample=freq,basefreq=62.5)

                _,_ = feature_extraction(clip_data)

                if 'features' in clip_data.keys():
                    D = clip_data['features']
                    featcols = list(D.columns)
                    D['Bradykinesia'] = subj_score['Bradykinesia'][trial]
                    D['Tremor'] = subj_score['Tremor'][trial]
                    D['Visit'] = session_list[trial]
                    D['Task'] = task
                    D['Subject'] = subj
                    Data = pd.concat([Data,D])
                    
                #clip_lens = []    
                
                #for c in range(len(clip_data['data'])):
                #    clip_lens.append(len(clip_data['data'][c]))
                #N = pd.DataFrame(data=np.asarray(clip_lens),columns=['Samples'],dtype='int')
                #N['Subject'] = subj
                #N['Task'] = task
                #N['Visit'] = session_list[trial]
                #numSamples = pd.concat([numSamples,N])
                                                
    cols = ['Subject','Visit','Task','Bradykinesia','Tremor'] + featcols
    Data = Data[cols]
    
    #numSamples = numSamples[['Subject','Visit','Task','Samples']]
    
    #if freq==62.5:
    #    numSamples.to_csv('Z:CIS-PD Study\\Sensor Sample Lengths.csv')
    
    return Data

In [10]:
# dictionary of side watch was worn on
subj_dict = {'Subject1':'right'}

task_list = ['Motor #1: Standing','Motor #2: Walking', 'Motor #3: Walking while Counting','Motor #4: Finger to Nose - Right',
             'Motor #4: Finger to Nose - Left', 'Motor #5: Alternating Hand Movements - Right',
             'Motor #5: Alternating Hand Movements - Left', 'Motor #6: Sit to Stand','Motor #7: Drawing on Paper',
             'Motor #8: Typing on a Computer','Motor #9: Nuts and Bolts','Motor #10: Drinking Water',
             'Motor #11: Organizing Folder','Motor #12: Folding Towels','Motor #13: Sitting']
task_scores = ['standing','walking','walking while counting','finger to nose right','finger to nose left','alternating right',
               'alternating left','sit to stand', 'drawing on a paper','typing on a computer keyboard',
               'assembling nuts and bolts','taking a glass of water and drinking','organizing sheets in a folder',
               'folding towels','sitting']
session_list = ['2 Weeks: Time 0', '2 Weeks: Time 30', '2 Weeks: Time 60', '2 Weeks: Time 90', '2 Weeks: Time 120', 
               '2 Weeks: Time 150', '4 Weeks']

In [11]:
for t in full_dict['Subject1'].keys():
    print(t,full_dict['Subject1'][t].keys())

Motor #1: Standing dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #2: Walking dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #3: Walking while Counting dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #4: Finger to Nose dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #5: Alternating Hand Movements dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #6: Sit to Stand dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #7: Drawing on Paper dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #8: Typing on a Computer dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #9: Nuts and Bolts dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #10: Drinking Water dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #11: Organizing Folder dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #12: Folding Towels dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #13: Sitting dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #4: Finger to Nose - Right dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #4: Finger to Nose - Left dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #5: Alternating Hand Movements - Right dict_keys([0, 1, 2, 3, 4, 5, 6])
Motor #5: Alternating Ha

In [12]:
for freq in [62.5,60,55,50,45,40,35,30,25,20,15,10,7.5,5]:
    print(freq)
    Data, AccelTime, GyroTime = DataAggregatorAccelGyro(full_dict,subj_dict,task_list,task_scores,session_list,freq)
    Data.to_csv('Accel+Gyro Data ' + str(freq) +  'Hz.csv')
    print(np.mean(np.vstack([np.array(a) for a in AccelTime[0]]),axis=0),
          np.mean(np.vstack([np.array(a) for a in GyroTime[0]]),axis=0))
    if freq==62.5:
        pickle.dump([AccelTime,GyroTime],open('SensorTimes.pkl','wb'))
    Data = DataAggregatorAccelOnly(full_dict,subj_dict,task_list,task_scores,session_list,freq)
    Data.to_csv('Accel Data ' + str(freq) +  'Hz.csv')

62.5
Subject 1004 (1 of 14)
Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
[0.00094972 0.00220122 0.00327984 0.03960924 0.00101311 0.008493
 0.00234325 0.0012781  0.01304899 0.00286176 0.00068771] [0.00094714 0.00219709 0.00328197 0.03936203 0.00101042 0.00847225
 0.00234366 0.00128039 0.01295865 0.00287563 0.00068146]
Subject 1004 (1 of 14)
Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
60
Subject 1004 (1 of 14)
Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)


Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
[0.00096726 0.00223091 0.00326861 0.01094791 0.00096668 0.00867659
 0.00236532 0.00130314 0.00348401 0.00291874 0.00069591] [0.00096428 0.00223366 0.00328506 0.01094894 0.00096296 0.00867401
 0.00236853 0.00129489 0.00349238 0.0029157  0.00069453]
Subject 1004 (1 of 14)
Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Subject 1029 (7 of 14)
Subject 1030 (8 of 14)
Subject 1032 (9 of 14)
Subject 1038 (10 of 14)
Subject 1044 (11 of 14)
Subject 1046 (12 of 14)
Subject 1049 (13 of 14)
Subject 1051 (14 of 14)
15
Subject 1004 (1 of 14)
Subject 1016 (2 of 14)
Subject 1018 (3 of 14)
Subject 1019 (4 of 14)
Subject 1020 (5 of 14)
Subject 1024 (6 of 14)
Sub